In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib3
import matplotlib.pyplot as plt
urllib3.disable_warnings()

In [18]:
def scrapper(df, url):
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, "html.parser")
    txt = soup.find_all('div', {'class' : 'team local pr mb10'})
    df_local = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])
    df_visitor = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])

    data = []

    local = url.split("/")[4]
    visitor = url.split("/")[5]
    for i in range(len(txt)):
        plchldr = txt[i].get_text('\t').split()
        while "" in plchldr:
            plchldr.replace(" ", "")
        data.append(plchldr)
    for i in range(len(data)):
        ind = i
        possible_events = ["Gol", "puerta", "Asistencia", "Entra", "Sale", "Amarilla", "Roja", "Lesionado", "anulado"]
        #print(data[i])

        while True:
            try:
                minute = int(data[ind][0].replace("'", ""))
                break
            except:
                ind -= 1
                pass
        player = None
        for j in range(len(data[i])):
                for event in possible_events:
                    if data[i][j] == event:
                        if event == "puerta":
                            event_type = "Gol en propia puerta"
                            player =  data[i][j-4]
                        elif event == "anulado":
                            event_type = "Gol anulado"
                            player =  data[i][j-2]
                        elif event == "Amarilla":
                            event_type = "Tarjeta amarilla"
                            player = data[i][j-2]
                        elif event == "Roja":
                            event_type = "Tarjeta roja"
                            player = data[i][j-2]
                        elif event == "Lesionado":
                            event_type = "Lesionada"
                        else:
                            event_type = event
                            player =  data[i][j-1]
        df_local = pd.concat([df_local, pd.DataFrame([{"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}])], ignore_index=True)

    txt = soup.find_all('div', {'class' : 'team visitor pr mb10'})
    data = []

    for i in range(len(txt)):
        plchldr = txt[i].get_text('\t').split()
        while "" in plchldr:
            plchldr.replace(" ", "")
        data.append(plchldr)

    for i in range(len(data)):
        ind = i
        possible_events = ["Gol", "puerta", "Asistencia", "Entra", "Sale", "Amarilla", "Roja", "Lesionado", "Tiro", "anulado"]
        while True:
            try:
                minute = int(data[ind][0].replace("'", ""))
                break
            except:
                ind -= 1
                pass
        for j in range(len(data[i])):
                for event in possible_events:
                    if data[i][j] == event:
                        if event == "puerta":
                            event_type = "Gol en propia puerta"
                            player =  data[i][j-4]
                        elif event == "anulado":
                            event_type = "Gol anulado"
                            player =  data[i][j-2]
                        elif event == "Amarilla":
                            event_type = "Tarjeta amarilla"
                            player = data[i][j-2]
                        elif event == "Roja":
                            event_type = "Tarjeta roja"
                            player = data[i][j-2]
                        elif event == "Lesionado":
                            event_type = "Lesionada"
                            player = data[i][j-1]
                        elif event == "Tiro":
                            event_type = "Tiro al palo"
                            player = data[i][j-1]
                        else:
                            event_type = event
                            player =  data[i][j-1]
        df_visitor = pd.concat([df_visitor, pd.DataFrame([{"Min": minute, "Event": event_type, "Player": player, "Team": visitor, "Against": local, "Home/Away": "Away"}])], ignore_index=True)

    return pd.concat([df, df_local, df_visitor])

#### Example of a single match:

In [19]:
df = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])
url = "https://ligaf.es/partido/villareal-cf-femenino/fc-barcelona-femenino/2023309307/eventos"
scrapper(df, url)

,Min,Event,Player,Team,Against,Home/Away
0,79,Entra,Sánchez,villareal-cf-femenino,fc-barcelona-femenino,Home
1,79,Sale,Ainoa,villareal-cf-femenino,fc-barcelona-femenino,Home
2,79,Entra,Bicho,villareal-cf-femenino,fc-barcelona-femenino,Home
3,79,Sale,García,villareal-cf-femenino,fc-barcelona-femenino,Home
4,66,Sale,Sheila,villareal-cf-femenino,fc-barcelona-femenino,Home
5,66,Entra,Estefa,villareal-cf-femenino,fc-barcelona-femenino,Home
6,56,Asistencia,Ainoa,villareal-cf-femenino,fc-barcelona-femenino,Home
7,56,Gol,García,villareal-cf-femenino,fc-barcelona-femenino,Home
8,55,Entra,Llompart,villareal-cf-femenino,fc-barcelona-femenino,Home
9,55,Sale,Cienfuegos,villareal-cf-femenino,fc-barcelona-femenino,Home


Another match:

In [20]:
df = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])
scrapper(df, "https://ligaf.es/partido/fc-barcelona-femenino/levante-las-planas-femenino/2023309436/eventos")

,Min,Event,Player,Team,Against,Home/Away
0,74,Gol,Mariona,fc-barcelona-femenino,levante-las-planas-femenino,Home
1,67,Sale,Torrejón,fc-barcelona-femenino,levante-las-planas-femenino,Home
2,67,Sale,Pina,fc-barcelona-femenino,levante-las-planas-femenino,Home
3,67,Entra,Bronze,fc-barcelona-femenino,levante-las-planas-femenino,Home
4,67,Entra,Aitana,fc-barcelona-femenino,levante-las-planas-femenino,Home
5,63,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home
6,58,Entra,Emma,fc-barcelona-femenino,levante-las-planas-femenino,Home
7,58,Sale,Crnogorcevic,fc-barcelona-femenino,levante-las-planas-femenino,Home
8,56,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home
9,48,Asistencia,Crnogorcevic,fc-barcelona-femenino,levante-las-planas-femenino,Home


#### Example of a whole team's season

In [21]:
page = requests.get("https://ligaf.es/equipo/fc-barcelona-femenino/8852/resultados/2023", verify=False)
soup = BeautifulSoup(page.content, "html.parser")
links = soup.find_all('a', {"class":"match-box ta-c"})
df = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])

for link in links:
        url = "https://ligaf.es" + str(link).split('"')[3] + "/eventos"
        #print(url)
        df = scrapper(df, url)

df

,Min,Event,Player,Team,Against,Home/Away
0,63,Entra,López,fc-barcelona-femenino,sc-huelva-femenino,Home
1,63,Sale,Geyse,fc-barcelona-femenino,sc-huelva-femenino,Home
2,63,Sale,Vilamala,fc-barcelona-femenino,sc-huelva-femenino,Home
3,63,Entra,Oshoala,fc-barcelona-femenino,sc-huelva-femenino,Home
4,63,Gol,Fernández,fc-barcelona-femenino,sc-huelva-femenino,Home
...,...,...,...,...,...,...
1,69,Sale,Vargas,sc-huelva-femenino,fc-barcelona-femenino,Away
2,69,Entra,López,sc-huelva-femenino,fc-barcelona-femenino,Away
3,69,Sale,Rós,sc-huelva-femenino,fc-barcelona-femenino,Away
4,46,Sale,Romero,sc-huelva-femenino,fc-barcelona-femenino,Away


#### Goals by a specific player

In [22]:
df.loc[(df["Event"]=="Gol") & (df["Player"]=="Oshoala")]

,Min,Event,Player,Team,Against,Home/Away
16,48,Gol,Oshoala,fc-barcelona-femenino,villareal-cf-femenino,Away
15,26,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Away
21,12,Gol,Oshoala,fc-barcelona-femenino,deportivo-alaves-femenino,Home
1,86,Gol,Oshoala,fc-barcelona-femenino,alhama-cf-femenino,Home
19,11,Gol,Oshoala,fc-barcelona-femenino,sevilla-fc-femenino-esp,Home
13,57,Gol,Oshoala,fc-barcelona-femenino,sc-huelva-femenino,Away
16,21,Gol,Oshoala,fc-barcelona-femenino,sc-huelva-femenino,Away
5,63,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home
8,56,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home
18,25,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home


#### Goals count by player of a team

In [23]:
df.loc[(df["Event"]=="Gol") & (df["Team"]=="fc-barcelona-femenino")]["Player"].value_counts()

Oshoala         19
Paralluelo      11
Pina            10
Aitana           9
Rolfö            9
Hansen           8
Crnogorcevic     7
Geyse            6
Mariona          6
Mapi             3
Bronze           3
Guijarro         2
López            2
Torrejón         2
Fernández        2
Engen            2
Codina           2
Paredes          1
Vilamala         1
Walsh            1
Name: Player, dtype: int64